In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./data/complaints.csv')

data.head()

,Consumer complaint narrative,Issue
0,My name is XXXX XXXX this complaint is not mad...,Incorrect information on your report
1,I searched on XXXX for XXXXXXXX XXXX and was ...,Fraud or scam
2,I have a particular account that is stating th...,Incorrect information on your report
3,I have not supplied proof under the doctrine o...,Attempts to collect debt not owed
4,Hello i'm writing regarding account on my cred...,Incorrect information on your report


In [3]:
data['Consumer complaint narrative'] = data['Consumer complaint narrative'].str.replace(r'\bX+[\sX]*\b', ' ', regex=True)

In [4]:
data['Issue_Code'] = data['Issue'].astype('category').cat.codes

In [5]:
data = data.rename(columns={'Consumer complaint narrative':'text'})

In [ ]:
X = data[['text']]
y = data['Issue_Code']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

In [ ]:
from collections import Counter
from scipy import sparse
from scipy.sparse.linalg import svds
import gensim

In [ ]:
review_counter = Counter()

for review in X_train['text']:
    review_counter.update(gensim.utils.simple_preprocess(review))

word_index = {word: i for i, word in enumerate(review_counter.keys())}
index_word = {i: word for i, word in enumerate(review_counter.keys())}

window_size = 2

cooccurrence_counter = Counter()

for review in X_train['text']:
    # First, tokenize the sentence
    sentence = gensim.utils.simple_preprocess(review)
    
    # Then, we'll build the window around each word
    for i, word in enumerate(sentence):
        window = sentence[max(0, i-2): i] + sentence[i+1: i+3]

        # Then, we'll up the counter value for that pair
        for other_word in window:
            cooccurrence_counter[(word, other_word)] += 1
            

# for word in review_counter.keys():
#     cooccurrence_counter[(word, word)] += review_counter[word]

In [ ]:
row_idx = []
col_idx = []
counts = []

for (word1, word2) in cooccurrence_counter.keys():
    row_idx.append(word_index[word1])
    col_idx.append(word_index[word2])
    counts.append(cooccurrence_counter[(word1, word2)])

cooccurrence_matrix = sparse.csc_matrix((counts, (row_idx, col_idx)), dtype = 'float')

dimension = 50

U, D, V = svds(cooccurrence_matrix, k = dimension)

word_vectors = U * D